In [ ]:
%load_ext tensorboard



import sys
import os

import config

# import Run_qlattice

import re
import hashlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import NN_run
import prepare_data as data_source
import mongodb_driver
import requests
import feyn

import utility_func as uf
import sklearn.metrics as skm 
from matplotlib import pyplot as plt
import numpy.ma as ma
from sklearn.model_selection import train_test_split

import rfet_model as rmdl



In [2]:
# mdb = mongodb_driver.mdb_connect("mdb_cmodel", "felixl", "mdbpass")
mdb = mongodb_driver.mdb_connect("mdb_cmodel", "root", "tyjWbtU4DRnuXqdhK")

mdb_col = mdb["NN"]

In [3]:
# PARAMETERS
minstep = 0.01                                  # minimum stepsize, refers to $\Delta V_{sweep}$ in section 4.2.1
i_max = 1                                       # maximum number of models to generate in parallel, with different seeds
str_run = "run_regular_grid_common_goal"        # PyTaurus run to extract the data from. "run_regular_grid_common_goal" is used in the dissertation
sweep_gate = "lat2"                             # the respective gate sweep to be used

# LINEAR MODEL

In [4]:


pars = {
    "data_type"                         : 'DC',
    "data_path"                         : f"{config.dir_data}/{str_run}_{sweep_gate}_minstep{minstep}_export.pkl",
    "save_path"                         : f"NN_models/mx/lin/",

    "mdl_type"                          : 'lin',
    "mdl_arch"                          : 'NN',

    # "augmentation_type"                 : "gauss_noise_3",
    "augmentation_type"                 : None,
    "train_test_split_random_seed"      : 20,

    "tf_epochs"                         : 1000,
    "tf_es_patience"                    : 120,
    "tf_es_delta"                       : 1E-07,
    "tf_n_neurons"                      : 16,
    
    "user"                              : os.getlogin(),
    "comment"                           : "minimal example test"
}

Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
Ylabel  =   ["ids"]



In [ ]:
### create MongoDB document

_id_lin = mdb_col.insert_one(pars)
_id_lin = str(_id_lin.inserted_id)
print(_id_lin)

In [ ]:
data = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=["ids"])

# select BG = lat1 = 0V
data = data[data["Vbglat1"] == 0]


res = []

# perform 3 times with a different random seed each time
for i in range(0,i_max):

    mdl,loss,MAE,R2 = NN_run.mx_run_NN_lin(
                    save_path = pars['save_path'], 
                    epochs = pars['tf_epochs'],
                    augmentation_type = pars['augmentation_type'],
                    random_seed = i,
                    data = data,
                    Xlabels = Xlabels,
                    Ylabel = Ylabel,
                    es_patience = pars["tf_es_patience"],
                    es_delta = pars["tf_es_delta"], 
                    _id = _id_lin,
                    n_neurons = pars["tf_n_neurons"])

    res.append({"i": i,"MAE": MAE, "R2": R2, "loss": loss, "model" : mdl})
    print(f"i: {i}\nloss {loss}\nMAE {MAE}\nR2 {R2}\nn")    


In [ ]:
# print results
pd_res_lin = pd.DataFrame(res)

pd_res_lin.sort_values(by=["R2"], inplace=True, ascending=False)
pd_res_lin.reset_index(inplace=True)

print(pd_res_lin)

# get best model
best_model_lin_id = pd_res_lin["R2"].idxmax()
best_model = pd_res_lin.loc[best_model_lin_id, "model"]


# save model with best R2 score

best_model.save(pars['save_path'] + "/" + str(_id_lin))


## Metrics

In [ ]:
# check availabel metrics for mdl

trainVal,test = uf.split_trainVal_test_set(all_data=data)
y_pred = mdl.predict(test[["Vlat21","Vfglat1", "Vtglat1"]])

y_pred = np.array(y_pred)
y_true = np.array(test["ids"])

print(f'R2 current model: %.6f' % skm.r2_score(y_true,y_pred))
print('MAE current model: %.4E' %(skm.mean_absolute_error(y_true,y_pred)))
print('MSE current model: %.4E' %(skm.mean_squared_error(y_true,y_pred)))
print('MAPE current model: %.4E' %(skm.mean_absolute_percentage_error(y_true,y_pred)))
print('sMAPE current model: %.6f' %(uf.Smape(y_true,y_pred)))


In [ ]:
Vfg = (data["Vfglat1"] == 1.4)
Vtg = (data["Vtglat1"] == 1.4)

Vbg = (data["Vbglat1"] == 0)

data_true = data.loc[Vfg & Vbg & Vtg]

fig, ax1= plt.subplots()

# ax2 = plt.twinx()

y_pred = mdl.predict(data_true[["Vlat21","Vfglat1", "Vtglat1"]])
                      
y_pred = y_pred[:,0]

ax1.plot(data_true["Vlat21"], - data_true["ids"], label="ids_true")
ax1.plot(data_true["Vlat21"], -y_pred, "o",label="ids_pred")



# ax2 = ax1.twinx()

# ax2.plot(cv_line["Vfglat1"], cv_line["Vfglat1"],".")

ax1.legend()
ax1.grid()


# LOGARITHMIC MODEL

In [ ]:
pars = {
    "data_type"                         : 'DC',
    "data_path"                         : f"{config.dir_data}/{str_run}_{sweep_gate}_minstep{minstep}_export.pkl",
    "save_path"                         : f"NN_models/mx/log/",

    "mdl_type"                          : 'log',
    "mdl_arch"                          : 'NN',

    "augmentation_type"                 : "gauss_noise_3",
    "train_test_split_random_seed"      : 2,

    "tf_epochs"                         : 1000,
    "tf_es_patience"                    : 120,
    "tf_es_delta"                       : 1E-07,
    
    "user"                              : os.getlogin(),
    "comment"                           : "minimal example test"
}

In [ ]:
# _id = "test"
_id_log = mdb_col.insert_one(pars)
_id_log = str(_id_log.inserted_id)
print(_id_log)

In [ ]:
data = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=["ids"])

# select BG = lat1 = 0V
data = data[data["Vbglat1"] == 0] 

# only regard i<1mA for logarithmic model, because for larger currents the linear model dominates anyway (weight function)
data = data[data["ids"] < 1]


Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]
Ylabel  =   ["ids"]

In [ ]:
res = []

for i in range(1,i_max):
    mdl,loss,MAE,R2 = NN_run.mx_run_NN_log(
                    save_path = pars['save_path'], 
                    epochs = pars['tf_epochs'],
                    augmentation_type = pars['augmentation_type'],
                    random_seed = i,
                    data = data,
                    Xlabels = Xlabels,
                    Ylabel = Ylabel,
                    es_patience = pars["tf_es_patience"],
                    es_delta = pars["tf_es_delta"], 
                    _id = _id_log)

    res.append({"i": i,"MAE": MAE, "R2": R2, "loss": loss, "model" : mdl})
    print(f"i: {i}\nloss {loss}\nMAE {MAE}\nR2 {R2}\nn")    


In [ ]:
# print results
pd_res_log = pd.DataFrame(res)

pd_res_log.sort_values(by=["R2"], inplace=True, ascending=False)
pd_res_log.reset_index(inplace=True)

print(pd_res_log)

# get best model
best_model_log_id = pd_res_log["R2"].idxmax()
best_model = pd_res_log.loc[best_model_log_id, "model"]


# save model with best R2 score

best_model.save(pars['save_path'] + "/" + str(_id_log))



# Q Model (symbolic regression)

In [ ]:
pars = {
    "data_type"                         : 'DC',
    "data_path"                         : f"{config.dir_data}/{str_run}_{sweep_gate}_minstep{minstep}_export.pkl",
    "save_path"                         : f"Qlattice/models/mx",

    "mdl_type"                          : 'Q',
    "mdl_arch"                          : 'SR',
    "augmentation_type"                 : None,

    "sr_num_epochs"                     : 500,
    "sr_max_complexity"                 : 50,
    "sr_train_size"                     : 0.6,
    "sr_transfer"                       : None,
    # "sr_transfer"                       : "Thung",

    
    "user"                              : os.getlogin(),
    "comment"                           : "minimal example test"
}



In [ ]:
_id_q = mdb_col.insert_one(pars)
_id_q = str(_id_q.inserted_id)
print(_id_q)

In [ ]:
if False:
    Q_list = ["Qfg", "Qtg", "Qlat2", "Qlat1"]

    data = data_source.load_dc_ramp_data(pars['data_path'], Xlabels =["Vlat21", "Vfglat1", "Vtglat1", "Vbglat1"], Ylabel=Q_list)

    # select BG = lat1 = 0V
    data = data[data["Vbglat1"] == 0]

    Xlabels =   ["Vlat21", "Vfglat1", "Vtglat1"]

    res = {}

    for Q in Q_list:
        
        Ylabel  =   [Q]

        (r2_all, MSE_all, train, test, best_model, mae_test, mse_test) = Run_qlattice.mx_run_ql(random_seed=1,
                                    num_epochs= pars["sr_num_epochs"],
                                    bound_ilat_to= None,
                                    sigma_gaus_aug= pars["augmentation_type"],
                                    max_complexity= pars["sr_max_complexity"],
                                    train_size = pars["sr_train_size"],
                                    transfer=pars["sr_transfer"],
                                    save_path=pars["save_path"],
                                    # starting_model=[feyn.Model.load('./Qlattice/models/Set2_transfer1.json')],
                                    starting_model=None,
                                    scale = True,
                                    data = data,
                                    Xlabels = Xlabels,
                                    Ylabel = Ylabel,
                                    sorting_crit='bic',
                                    id = _id_q + "_" + Q ,
                                    # id = "qfg_test",
                                    mdl_type = pars["mdl_type"],
                                    scaling= 1E15
                                    )
        
        res[Q] = {"r2_all" : r2_all, "MSE_all" : MSE_all, "train": train, "test" : test, "model" : best_model, "mae_test" : mae_test, "mse_test" : mse_test}
else:
    _id_q = "650375e8daf354a0227a9917"  #use an existing q model

# COMPOSE MODEL

In [ ]:
# instantiate RFET compact model object from linear, logarithmic and Q model
compact_mdl = rmdl.rfet_model("testnamex", I_model = {"lin" : _id_lin, "log" : _id_log}, Q_model = _id_q)     

# create Verilog-A file in <path>
compact_mdl.compile(path="export/veriloga.va", significant_digits = 8) 

# SIMULATE INVERTER IN CADENCE

In [ ]:
compact_mdl.sim_cds_inv(template_file="templates/inv_template.ocn", n_steps=200)

In [ ]:
print(f"{_id_lin}, {_id_log}")

In [ ]:
# get transition from 90% to 10% of Vdd
y_high  = 0.9 * 1.4
y_low   = 0.1 * 1.4

xlow, xhigh, width, xcenter, df_extended, fig_inv = rmdl.rfet_model.getTransition(compact_mdl.trace_cds_inv, y_high, y_low, plot=True)

In [ ]:
rmdl.rfet_model.plotINVVTC(compact_mdl.trace_cds_inv,plot=True)